# Main Extraction Engine. Run as many times as needed to get full list of players. 

In [11]:
# import python libraries and nba library
import pandas as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging
import os
import glob

In [12]:
# Get all files in working data directory
path = os.getcwd()
path = path.replace("notebooks","data")
csv_files = glob.glob(os.path.join(path,"*.csv"))

In [13]:
check_df = pd.DataFrame()
for file in csv_files:

    df = pd.read_csv(file)
    frames = [check_df, df]
    check_df = pd.concat(frames)

In [14]:
unique_ids = check_df.PLAYER_ID.unique()
print(unique_ids)

[1631260  201980  203110  203210 1630224 1629750  201145 1630182 1631100
  201933 1629656 1629060 1630169 1631160 1629788 1630181  203501  201935
 1630702 1631199 1626149  203914 1630223  203925 1630284  202699 1627885
 1628404 1628392    2617 1630573 1629637 1630165  202330 1627823 1626195
 1629639 1627741 1629312  201588 1630792 1630207 1628988  201950  203200
 1626158  201143 1629659 1627863 1631216 1631309 1628989 1630643 1630624
 1629631 1630538  201586    2738  204060 1627742  202681 1628371 1631093
 1630543 1628367 1628382  202704 1628991    2544 1629610 1629660 1630198
 1628993 1629661 1630552  201949 1629640 1630553 1626169  203999 1630637
 1627745 1630529 1630539 1630200 1626145 1627884  201599  202709 1631107
 1629662 1626163 1631211 1628379 1631117 1630296 1630557 1628467 1630233
 1628995 1631132 1629723 1627788 1628436 1630249 1630228 1628398 1631222
  203897 1630237 1629111 1629641 1630639 1627774 1627747 1627814  203458
  202695 1630184  203081 1629642 1630587 1631254 16

In [15]:
# import teams, players, and sort them into active or all players
from nba_api.stats.static import teams, players
nba_players = players.get_players()
active_players = players.get_active_players()
teams = teams.get_teams()

In [16]:
# import use-case specific functions
from nba_api.stats.endpoints import playercareerstats

In [17]:
# Add entry for each player on whether they have been analyzed or not. 
# Default is not analyzed
for player in active_players:
    player['Parsed ?'] = False
    if player["id"] in unique_ids:
        player["Parsed ?"] = True

In [18]:
# Initialize collection dataframe
master_list = pd.DataFrame()

# Create temp dataframe and initialize count to serve as a staging 
# Dataframe and break up the merging into small batches
temp_df = pd.DataFrame()
count = 0

In [19]:
# Parse thru each active player
first_id = 0
last_id = 0
try:
    for player in active_players:

        # Check if they have been already analyzed
        if player['Parsed ?'] == True:
            pass
        
        # If they haven't, proceed
        else:
            passedId = player['id']
            player_name = player["full_name"]

            # Get player stats based on their ID
            career_stats = playercareerstats.PlayerCareerStats(player_id = passedId).get_data_frames()[0]
            
            # Get ID of first and last players analyzed for file naming purposes
            if first_id == 0:
                first_id = passedId
            last_id = passedId

            # Add player names to resulting dataframe to keep track of players
            career_stats.insert(1,"full_name",player_name)

            # Append resulting dataframe to temp dataframe
            frames = [temp_df,career_stats]
            temp_df = pd.concat(frames)

            # Set analyzed flag to true
            print(player_name)
            player['Parsed ?'] = True
            count = count + 1

            # Once enough volume is reached in the temp dataframe, append it to the main and re-initialize the dataframe
            if (count%20) == 0:
                frames = [master_list, temp_df]
                master_list = pd.concat(frames)
                temp_df = pd.DataFrame()
                print("Temp Full - Resetting")
except:
    print("TimeoutError")

LiAngelo Ball
Jules Bernard
Marcus Bingham Jr.
Jamaree Bouyea
Gabe Brown
Jalen Crutcher
Darius Days
Anthony Duruji
Abdul Gaddy
Collin Gillespie
Grant Golden
RaiQuan Gray
Mouhamadou Gueye
Chet Holmgren
Matt Hurt
TimeoutError


In [20]:
if first_id == 0:
    pass
else:
    master_list.to_csv("C:/Github Repo/Mario-Jupyter-Codespace/data/basketballProject/nba_data_"+str(first_id)+"to"+str(last_id)+".csv")